In [1]:
from dataclasses import dataclass
from datetime import datetime

import sys
sys.executable


'/home/xyz/projects/ctc-filter/.venv/bin/python'

In [2]:
import talib as ta
import pandas as pd
import numpy as np

In [3]:
from pandas import DataFrame

In [4]:
def get_data():
    with open('YGG-USDT-SWAP-1H.json') as f:
        data = pd.read_json(f)
    return data

def to_candles(data: list) -> DataFrame:
    """将数据转换为用于绘制 K 线图的 DataFrame

    Args:
        data: JSON 数据

    Returns:
        DataFrame: 一组 K 线数据帧
    """
    df = pd.DataFrame(
        data,
        columns=[
            "ts",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "volCcy",
            "volCcyQuote",
            "_",
        ],
    )
    return df

In [5]:
df = to_candles(get_data())
df["avg"] = (df["high"] + df["low"] + df["close"]) / 3
# ts -> datetime
df["datetime"] = pd.to_datetime(df["ts"], unit="ms")
df

,ts,open,high,low,close,volume,volCcy,volCcyQuote,_,avg,datetime
0,1717142400000,0.9697,0.9825,0.9681,0.9768,1188739,1188739,1.160733e+06,1,0.975800,2024-05-31 08:00:00
1,1717146000000,0.9767,0.9829,0.9714,0.9763,809882,809882,7.906980e+05,1,0.976867,2024-05-31 09:00:00
2,1717149600000,0.9761,0.9820,0.9635,0.9685,1119119,1119119,1.085977e+06,1,0.971333,2024-05-31 10:00:00
3,1717153200000,0.9688,0.9761,0.9661,0.9686,782579,782579,7.598807e+05,1,0.970267,2024-05-31 11:00:00
4,1717156800000,0.9685,0.9827,0.9635,0.9746,1851669,1851669,1.804400e+06,1,0.973600,2024-05-31 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...
96,1717488000000,1.1309,1.1395,1.1277,1.1381,1100781,1100781,1.248468e+06,1,1.135100,2024-06-04 08:00:00
97,1717491600000,1.1379,1.1545,1.1355,1.1371,2405240,2405240,2.750482e+06,1,1.142367,2024-06-04 09:00:00
98,1717495200000,1.1376,1.1594,1.1375,1.1509,2201353,2201353,2.530906e+06,1,1.149267,2024-06-04 10:00:00
99,1717498800000,1.1508,1.1606,1.1300,1.1389,2757728,2757728,3.159867e+06,1,1.143167,2024-06-04 11:00:00


In [6]:
MFI_LEN = 7
STOCH_K = 5
STOCH_D = 2
SMOOTH_LEN = 1.75
STOCH_WEIGHT = 0.4
OVERBOUGHT = 60.0
EXTEND_MULT = 1
MFI_WEIGHT = 0.4

# 背离

In [7]:
MAX_ARR_PIV_SIZE = 30
MAX_ARR_LINE_SIZE = 250
PIV_LEN_LEFT = 7 
PIV_LEN_RIGHT = 1 
MIN_PEAK_DIST = 7
MAX_PEAK_DIST = 80

In [8]:
# 计算枢轴点
df['ph'] = ta.MAX(df['high'], timeperiod=PIV_LEN_LEFT + PIV_LEN_RIGHT + 1)
df['pl'] = ta.MIN(df['low'], timeperiod=PIV_LEN_LEFT + PIV_LEN_RIGHT + 1)
df['phI'] = ta.MAX(df['avg'], timeperiod=PIV_LEN_LEFT + PIV_LEN_RIGHT + 1)
df['plI'] = ta.MIN(df['avg'], timeperiod=PIV_LEN_LEFT + PIV_LEN_RIGHT + 1)
df

,ts,open,high,low,close,volume,volCcy,volCcyQuote,_,avg,datetime,ph,pl,phI,plI
0,1717142400000,0.9697,0.9825,0.9681,0.9768,1188739,1188739,1.160733e+06,1,0.975800,2024-05-31 08:00:00,NaN,NaN,NaN,NaN
1,1717146000000,0.9767,0.9829,0.9714,0.9763,809882,809882,7.906980e+05,1,0.976867,2024-05-31 09:00:00,NaN,NaN,NaN,NaN
2,1717149600000,0.9761,0.9820,0.9635,0.9685,1119119,1119119,1.085977e+06,1,0.971333,2024-05-31 10:00:00,NaN,NaN,NaN,NaN
3,1717153200000,0.9688,0.9761,0.9661,0.9686,782579,782579,7.598807e+05,1,0.970267,2024-05-31 11:00:00,NaN,NaN,NaN,NaN
4,1717156800000,0.9685,0.9827,0.9635,0.9746,1851669,1851669,1.804400e+06,1,0.973600,2024-05-31 12:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1717488000000,1.1309,1.1395,1.1277,1.1381,1100781,1100781,1.248468e+06,1,1.135100,2024-06-04 08:00:00,1.1526,1.0919,1.135567,1.112900
97,1717491600000,1.1379,1.1545,1.1355,1.1371,2405240,2405240,2.750482e+06,1,1.142367,2024-06-04 09:00:00,1.1545,1.0919,1.142367,1.112900
98,1717495200000,1.1376,1.1594,1.1375,1.1509,2201353,2201353,2.530906e+06,1,1.149267,2024-06-04 10:00:00,1.1594,1.0919,1.149267,1.112900
99,1717498800000,1.1508,1.1606,1.1300,1.1389,2757728,2757728,3.159867e+06,1,1.143167,2024-06-04 11:00:00,1.1606,1.0919,1.149267,1.112900


In [9]:
@dataclass
class Pivot:
    price: float
    ndx: int
    indicator: float
    datetime: datetime

@dataclass
class DiverLine:
    def __init__(self, ln, lb):
        self.ln = ln
        self.lb = lb

pivHigh: list[Pivot] = []
pivLow: list[Pivot] = []
bullDivers = []
bearDivers = []
bullHidDivers = []
bearHidDivers = []


## 添加枢轴点函数
实现添加枢轴点的函数。

In [10]:
def add_pivot_high(new_piv):
    if len(pivHigh) > 0:
        prev = pivHigh[0]
        if prev.price < new_piv.price and prev.indicator < new_piv.indicator:
            pivHigh.pop(0)
    pivHigh.insert(0, new_piv)
    if len(pivHigh) > MAX_ARR_PIV_SIZE:
        pivHigh.pop()

def add_pivot_low(new_piv):
    if len(pivLow) > 0:
        prev = pivLow[0]
        if prev.price > new_piv.price and prev.indicator > new_piv.indicator:
            pivLow.pop(0)
    pivLow.insert(0, new_piv)
    if len(pivLow) > MAX_ARR_PIV_SIZE:
        pivLow.pop()

## 检查指标值函数

实现检查指标值是否满足条件的函数。

In [11]:
def is_greater_ind_value(start, end, max_ind_val, all_ind_val):
    for i in range(start, end + 1):
        if all_ind_val[i] > max_ind_val:
            return False
    return True

def is_lower_ind_value(start, end, min_ind_val, all_ind_val):
    for i in range(start, end + 1):
        if all_ind_val[i] < min_ind_val:
            return False
    return True

## 检查高低点函数

实现检查高低点是否满足条件的函数。

In [12]:
def is_greater_high(start, end, point_high, all_highs):
    """更高的高点"""
    for i in range(start, end + 1):
        if all_highs[i] > point_high:
            return False
    return True

def is_lower_low(start, end, point_low, all_lows):
    """更低的低点"""
    for i in range(start, end + 1):
        if all_lows[i] < point_low:
            return False
    return True

In [107]:
def search_bear_div(new_piv):
    result = False
    last_ind_high = new_piv.indicator
    if last_ind_high > 0:
        for piv in pivHigh:
            if new_piv.ndx - piv.ndx >= MAX_PEAK_DIST:
                break
            if piv.price > new_piv.price:
                break
            if piv.indicator > last_ind_high:
                last_ind_high = piv.indicator
            if piv.price <= new_piv.price and piv.indicator > new_piv.indicator and piv.indicator >= last_ind_high and (new_piv.ndx - piv.ndx >= MIN_PEAK_DIST) and piv.indicator > 50:
                if is_greater_ind_value(piv.ndx, new_piv.ndx, piv.indicator) and is_greater_high(piv.ndx, new_piv.ndx, new_piv.price):
                    # draw_new_div(piv, new_piv, False)
                    result = True
                    break
    return result

def search_bull_div(new_piv):
    result = False
    new_ind_low = new_piv.indicator
    if new_ind_low < 0:
        for piv in pivLow:
            if new_piv.ndx - piv.ndx >= MAX_PEAK_DIST:
                break
            if piv.price < new_piv.price:
                break
            if piv.indicator < new_ind_low:
                new_ind_low = piv.indicator
            if piv.price >= new_piv.price and piv.indicator < new_piv.indicator and piv.indicator <= new_ind_low and (new_piv.ndx - piv.ndx >= MIN_PEAK_DIST) and piv.indicator < -50:
                if is_lower_ind_value(piv.ndx, new_piv.ndx, piv.indicator) and is_lower_low(piv.ndx, new_piv.ndx, new_piv.price):
                    # draw_new_div(piv, new_piv, True)
                    result = True
                    break
    return result


In [108]:
# 处理数据
for i in range(len(df)):
    if not np.isnan(df['ph'].iloc[i]):
        last_ph = Pivot(df['ph'].iloc[i], i - PIV_LEN_RIGHT, df['phI'].iloc[i], df['datetime'].iloc[i])
    if not np.isnan(df['pl'].iloc[i]):
        last_pl = Pivot(df['pl'].iloc[i], i - PIV_LEN_RIGHT, df['plI'].iloc[i], df['datetime'].iloc[i])
    
    if not np.isnan(df['phI'].iloc[i]):
        sig = df['phI'].iloc[i]
        sig_ndx = i - PIV_LEN_RIGHT
        if sig_ndx - last_ph.ndx <= 2:
            new_piv = Pivot(last_ph.price, sig_ndx, sig, df['datetime'].iloc[i])
            added = search_bear_div(new_piv)
            add_pivot_high(new_piv)
            if added:
                print(f"bearish divergence at {new_piv.ndx}")
    
    if not np.isnan(df['plI'].iloc[i]):
        sig = df['plI'].iloc[i]
        sig_ndx = i - PIV_LEN_RIGHT
        if sig_ndx - last_pl.ndx <= 2:
            new_piv = Pivot(last_pl.price, sig_ndx, sig, df['datetime'].iloc[i])
            added = search_bull_div(new_piv)
            add_pivot_low(new_piv)
            if added:
                print(f"bullish divergence at {new_piv.ndx}")

In [109]:
pivHigh

[Pivot(price=1.1606, ndx=99, indicator=1.1492666666666667, datetime=Timestamp('2024-06-04 12:00:00')),
 Pivot(price=1.1606, ndx=98, indicator=1.1492666666666667, datetime=Timestamp('2024-06-04 11:00:00')),
 Pivot(price=1.1594, ndx=97, indicator=1.1492666666666667, datetime=Timestamp('2024-06-04 10:00:00')),
 Pivot(price=1.1526, ndx=94, indicator=1.1355666666666666, datetime=Timestamp('2024-06-04 07:00:00')),
 Pivot(price=1.1526, ndx=93, indicator=1.1353, datetime=Timestamp('2024-06-04 06:00:00')),
 Pivot(price=1.1526, ndx=92, indicator=1.1353, datetime=Timestamp('2024-06-04 05:00:00')),
 Pivot(price=1.1526, ndx=91, indicator=1.1353, datetime=Timestamp('2024-06-04 04:00:00')),
 Pivot(price=1.1526, ndx=90, indicator=1.1353, datetime=Timestamp('2024-06-04 03:00:00')),
 Pivot(price=1.1526, ndx=89, indicator=1.1353, datetime=Timestamp('2024-06-04 02:00:00')),
 Pivot(price=1.1526, ndx=88, indicator=1.1352666666666666, datetime=Timestamp('2024-06-04 01:00:00')),
 Pivot(price=1.1408, ndx=86, i

In [110]:
pivLow

[Pivot(price=1.0919, ndx=99, indicator=1.1160666666666668, datetime=Timestamp('2024-06-04 12:00:00')),
 Pivot(price=1.0919, ndx=98, indicator=1.1129, datetime=Timestamp('2024-06-04 11:00:00')),
 Pivot(price=1.0919, ndx=97, indicator=1.1129, datetime=Timestamp('2024-06-04 10:00:00')),
 Pivot(price=1.0919, ndx=96, indicator=1.1129, datetime=Timestamp('2024-06-04 09:00:00')),
 Pivot(price=1.0919, ndx=95, indicator=1.1129, datetime=Timestamp('2024-06-04 08:00:00')),
 Pivot(price=1.0919, ndx=94, indicator=1.1129, datetime=Timestamp('2024-06-04 07:00:00')),
 Pivot(price=1.0919, ndx=93, indicator=1.1129, datetime=Timestamp('2024-06-04 06:00:00')),
 Pivot(price=1.0919, ndx=92, indicator=1.1129, datetime=Timestamp('2024-06-04 05:00:00')),
 Pivot(price=1.0919, ndx=91, indicator=1.1129, datetime=Timestamp('2024-06-04 04:00:00')),
 Pivot(price=1.101, ndx=90, indicator=1.1129, datetime=Timestamp('2024-06-04 03:00:00')),
 Pivot(price=1.113, ndx=88, indicator=1.1195333333333333, datetime=Timestamp('2